In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
#Nos conectamos al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def crear_diccionario(dmel_cv):
    grupo = dmel_cv.groupby('Model_nodos')['Symbol'].first().reset_index()
    grupo = grupo.rename(columns={'Symbol': 'ID_nuevo'})
    dmel_cv = dmel_cv.merge(grupo, on='Model_nodos', how='left')
    diccionario = dmel_cv[['Astyanax_ortologos', 'Symbol', 'ID_nuevo']].drop_duplicates()
    return diccionario

def actualizar_diccionario(nuevo_df, diccionario):
    grupo = nuevo_df.groupby('Model_nodos')['Symbol'].first().reset_index()
    grupo = grupo.rename(columns={'Symbol': 'ID_nuevo'})
    nuevo_df = nuevo_df.merge(grupo, on='Model_nodos', how='left')

    conocidos = diccionario[['Astyanax_ortologos', 'ID_nuevo']]
    df_match = nuevo_df.merge(conocidos, on='Astyanax_ortologos', how='inner', suffixes=('', '_previo'))
    df_match['ID_nuevo'] = df_match['ID_nuevo_previo']
    df_match = df_match.drop(columns=['ID_nuevo_previo'])

    df_no_match = nuevo_df[~nuevo_df['Astyanax_ortologos'].isin(conocidos['Astyanax_ortologos'])]
    df_final = pd.concat([df_match, df_no_match], ignore_index=True)

    nuevo_diccionario = df_final[['Astyanax_ortologos', 'Symbol', 'ID_nuevo']].drop_duplicates()
    diccionario_actualizado = pd.concat([diccionario, nuevo_diccionario], ignore_index=True)
    diccionario_actualizado = diccionario_actualizado.drop_duplicates(subset=['Astyanax_ortologos'])
    return diccionario_actualizado

In [ ]:
dmel_cv = pd.read_csv("/content/drive/My Drive/Sakura/Diccionarios/cv_Dmel_Isoformas.csv")
homo_cv = pd.read_csv("/content/drive/My Drive/Sakura/Diccionarios/cv_Homo_Isoformas.csv")
mus_cv  = pd.read_csv("/content/drive/My Drive/Sakura/Diccionarios/cv_Mus_Isoformas.csv")

In [ ]:
diccionario_df = crear_diccionario(dmel_cv)

In [ ]:
diccionario_df = actualizar_diccionario(homo_cv, diccionario_df)

In [ ]:
diccionario_df = actualizar_diccionario(mus_cv, diccionario_df)

In [ ]:
diccionario_df.shape

(4628, 3)

In [ ]:
output_path = "/content/drive/My Drive/Sakura/Diccionarios/CV_diccionario.csv"
diccionario_df.to_csv(output_path, index=False)